In [1]:
!pip install yfinance

  Preparing metadata (setup.py) ... - done
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=6bb3b0fb3edecf56834955025f25a0df83311d9f601a299989cc3ee99ec6f0cc
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking


In [2]:
!pip install fbprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=9429442 sha256=49b7381ccdf6b52952959eff88938d1c1dcfa4c1c6354a0e5e5febde48642dae
  Stored in directory: /root/.cache/pip/wheels/cd/a1/12/db63ff624de492fe6cccf676091a0860fdde2ffde4bc3280e2
Successfully built fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.68
    Uninstalling cmdstanpy-0.9.68:
      Successfully uninstalled cmdstanpy-0.9.68
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.0.1 requires cmdstanpy==0.9.68, but you have cmdstanpy 0.9.5 which is incompatible.


In [3]:
# essentials
import numpy as np 
import pandas as pd 
import math
import datetime
from datetime import datetime as dt
import yfinance as yf
# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet.diagnostics import cross_validation
from fbprophet.plot import plot_cross_validation_metric
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
# time series algorithm
from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
# reproducibility
np.random.seed(34)

# Jupyter magic
%reload_ext autoreload
%autoreload 2
%matplotlib inline

sns.set()

In [4]:

# class model_prophet:
#     def get_data(self):
#         data_usdrub = yf.download('USDRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
#         data_eurub = yf.download('EURRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
#         return data_usdrub,data_eurub
#     def ml_model(self,df):
#         train_columns = ['Close']
#         train = df[train_columns]
#         train = train.rename(columns={'Close':'y'})
#         train.index.names =['ds']
#         train =train.reset_index()
#         df = df.rename(columns={'Close':'y'})
#         df.index.names =['ds']
#         df =train.reset_index()
    
#         m = Prophet(daily_seasonality=False,yearly_seasonality=True,
#             weekly_seasonality= True,changepoint_prior_scale= 0.05, 
#             seasonality_prior_scale= 20.0, holidays_prior_scale= 5.0)
#         m.fit(train)
    
#         future = m.make_future_dataframe(periods=30)
#         forecast = m.predict(future)
    
#         return forecast
#     def get_forecast(self,df):
#         forecast = ml_model(df)
#         forecast = forecast[['ds','yhat_lower']][-30:]
#         return forecast
#     def ml_predict(self,usd=True,eur=True):
#         df_usdrub,df_eurub=get_data()
#         if usd==True:
#             forecast_usdrub = get_forecast(df_usdrub)
#             return forecast_usdrub
#         if eur==True:
#             forecast_eurub = get_forecast(df_eurub)
#             return forecast_eurub

In [5]:
def get_data():
    data_usdrub = yf.download('USDRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
    data_eurub = yf.download('EURRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
    return data_usdrub,data_eurub

In [6]:
def ml_model(df):
    df_columns = ['ds','y']
    train_columns = ['Close']
    train = df[train_columns]
    train = train.rename(columns={'Close':'y'})
    train.index.names =['ds']
    train =train.reset_index()
    df = df.rename(columns={'Close':'y'})
    df.index.names =['ds']
    df =train.reset_index()
    
    m = Prophet(daily_seasonality=False,yearly_seasonality=True,
            weekly_seasonality= True,changepoint_prior_scale= 0.05, 
            seasonality_prior_scale= 20.0, holidays_prior_scale= 5.0)
    m.fit(train)
    
    future = m.make_future_dataframe(periods=30)
    forecast = m.predict(future)
    
    cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df[df_columns].set_index('ds'))
    cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
    mae=np.mean(abs(cmp_df[-45:-15]['e']))
    
    forecast = forecast[['ds','yhat_lower']][-30:]
    forecast['yhat_lower'] = forecast['yhat_lower'].apply(lambda x:x-mae)
    return forecast
    
    

In [7]:
def get_forecast(df):
    forecast = ml_model(df)
    forecast = forecast[['ds','yhat_lower']][-30:]
    return forecast

In [8]:
def ml_predict(usd=True,eur=True):
    df_usdrub,df_eurub=get_data()
    if usd==True:
        forecast_usdrub = get_forecast(df_usdrub)
        return forecast_usdrub
    if eur==True:
        forecast_eurub = get_forecast(df_eurub)
        return forecast_eurub
        
    

In [9]:
usdp = ml_predict(eur=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Initial log joint probability = -8.2554
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       3921.59    0.00597789       444.714           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     196       3929.71   9.97565e-05       120.412   2.884e-07       0.001      278  LS failed, Hessian reset 
     199       3930.02    0.00591473        385.25           1           1      282   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        3933.4   0.000128731       198.896   3.213e-07       0.001      452  LS failed, Hessian reset 
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       3934.75     0.0038171       139.958       1

In [10]:
usdp

,ds,yhat_lower
1405,2022-05-26,58.897167
1406,2022-05-27,59.446905
1407,2022-05-28,61.464389
1408,2022-05-29,61.669003
1409,2022-05-30,59.521899
1410,2022-05-31,59.782689
1411,2022-06-01,59.993165
1412,2022-06-02,59.848271
1413,2022-06-03,59.969022
1414,2022-06-04,61.945234


In [11]:
data_usdrub = yf.download('USDRUB=X', start='2017-01-01', end=datetime.datetime.now().date())
data_usdrub

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-02,61.231300,61.545898,61.155399,61.231300,61.231300,0
2017-01-03,61.228298,61.228298,60.332401,61.228298,61.228298,0
2017-01-04,60.923000,61.145802,60.471100,60.923000,60.923000,0
2017-01-05,60.363899,60.409500,59.152199,60.365002,60.365002,0
2017-01-06,59.401199,59.721600,59.091400,59.401199,59.401199,0
...,...,...,...,...,...,...
2022-05-19,64.214996,64.714996,61.615002,64.214996,64.214996,0
2022-05-20,62.090000,63.465000,57.889900,62.090000,62.090000,0
2022-05-23,62.465000,62.465000,57.188000,62.465000,62.465000,0


In [12]:
data_eurub = yf.download('EURRUB=X', start='2017-01-01', end=datetime.datetime.now().date())

[*********************100%***********************]  1 of 1 completed
